In [3]:
!pip install pinecone sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 6.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 5.7 MB/s eta 0:00:00


In [13]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [4]:
!pip install --upgrade pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.0.0
    Uninstalling pinecone-plugin-inference-3.0.0:
      Successfully uninstalled pinecone-plugin-inference-3.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.1 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [5]:
!ls /kaggle/input/transnormerllm

config.json		special_tokens_map.json   tokenizer_config.json
generation_config.json	tokenization_baichuan.py
pytorch_model.bin	tokenizer.model


In [6]:
!pip install einops triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.3 MB/s eta 0:00:00:00:0100:01


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Define the model path
model_path = "/kaggle/input/transnormerllm"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

configuration_transnormer.py:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- configuration_transnormer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_transnormer.py:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

utils.py:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

srmsnorm_triton.py:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- srmsnorm_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


norm.py:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- utils.py
- srmsnorm_triton.py
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


lightning_attention2.py:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- lightning_attention2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- modeling_transnormer.py
- utils.py
- lightning_attention2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [8]:
# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1

rag_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    device=device
)

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return the GPU name if available

True
Tesla T4


In [9]:
# Test the pipeline
output = rag_pipeline("Once upon a time", max_length=50, num_return_sequences=1)
print(output)

Both `max_new_tokens` (=2048) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Once upon a time, before the fall, the Earth had several primary continents. The largest of these was Africa, being the largest landmass and being home to most of its inhabitants. Africa had many small island nations as well, and it has been described as the cradle of mankind. The continent was primarily inhabited by a multitude of native peoples, ranging in size from the Congo to South Sudan, some of whom are now part of Africa as states today.\nQuestion: did africa have a continents?\nAnswer: True'}]


In [10]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

# Initialize Pinecone client
pc = Pinecone(
    api_key="pcsk_5nP68u_U7MeWfAo2hKpwwkmWLyAC2VNEuCgZo8EQzs4Zox7yyrdUAXjM5rttShTEEGAPuZ"
)

# Define the index name
index_name = "chrome-history-index"

# Check if the index exists
existing_indexes = pc.list_indexes().names()  # Get list of existing indexes
if index_name not in existing_indexes:
    raise ValueError(f"Index '{index_name}' does not exist. Please verify the name or create it in the Pinecone dashboard.")

# Access the Pinecone index
index = pc.Index(index_name)

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
def perform_filtered_query(query, filter_conditions=None, query_vector=None):
    """
    Perform a Pinecone query with combined filters and augment the result for Chrome history.
    """
    if filter_conditions is None:
        filter_conditions = {}

    # Use query_vector for title-based search, or default vector for filter-only queries
    if query_vector is None:
        query_vector = [0.0] * 384  # Dummy vector for metadata-only filtering

    # Query Pinecone with combined filters
    results = index.query(
        vector=query_vector,
        top_k=5,
        include_metadata=True,
        filter=filter_conditions
    )

    # Construct context with titles and timestamps, filtered by domain or time
    context = "\n".join([
        f"{res['metadata']['Title']} ({res['metadata']['Timestamp']})"
        for res in results["matches"]
    ])

    # Limit context length to avoid exceeding model limits
    max_context_length = 300  # Adjust based on model's input size
    truncated_context = context[:max_context_length]

    # Augment the query with the context
    augmented_query = f"Here are some results for your query:\n{truncated_context}\n\nQuery: {query}"
    
    # Debug: Print the augmented query
    print(f"Augmented Query:\n{augmented_query}\n")

    # Generate response
    response = rag_pipeline(augmented_query, max_new_tokens=100, num_return_sequences=1)
    
    # Handle empty or nonsensical responses
    if response and response[0]["generated_text"].strip():
        return response[0]["generated_text"]
    else:
        return "No relevant entries found for the specified query or time range."

In [26]:
import re

def perform_rag_query(query):
    """
    Perform a RAG query by detecting query type and routing to the appropriate handler.
    Supports combined queries (date + domain + title).
    """
    filter_conditions = {}
    query_vector = None

    # Detect date in query
    date_match = re.search(r'\b(\d{4}-\d{2}-\d{2})\b', query, re.IGNORECASE)
    if date_match:
        filter_conditions["Timestamp"] = date_match.group(1)
    
    # Detect domain in query
    domain_match = re.search(r'\b(?:www\.)?([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})\b', query, re.IGNORECASE)
    if domain_match:
        filter_conditions["Domain"] = domain_match.group(1)
    
    # Detect title keywords (if no explicit date/domain filter)
    if len(filter_conditions) == 0 or "title" in query.lower():
        query_vector = embedding_model.encode(query).tolist()

    # Pass combined filters and query vector to `perform_filtered_query`
    return perform_filtered_query(query, filter_conditions=filter_conditions, query_vector=query_vector)

In [27]:
# Example usage
query = "List all the movies i watched of govinda"
response = perform_rag_query(query)
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Augmented Query:
Here are some results for your query:
(4) Coolie No. 1 | All Comedy Scenes | Govinda | Karishma Kapoor | Pooja Films - YouTube (2024-04-05 20:21:09)
govinda hululu - Google Search (2024-07-27 05:38:55)
govinda hululu - Google Search (2024-07-27 05:38:41)
(5) Hungama - Part 7 | Paresh Rawal, Rajpal Yadav & Akshaye Khanna | Hindi Movies |

Query: List all the movies i watched of govinda

Here are some results for your query:
(4) Coolie No. 1 | All Comedy Scenes | Govinda | Karishma Kapoor | Pooja Films - YouTube (2024-04-05 20:21:09)
govinda hululu - Google Search (2024-07-27 05:38:55)
govinda hululu - Google Search (2024-07-27 05:38:41)
(5) Hungama - Part 7 | Paresh Rawal, Rajpal Yadav & Akshaye Khanna | Hindi Movies |

Query: List all the movies i watched of govinda kumar ji, and their directors from 1985 to 1989.

Answer: There are several movies that govinda kumar ji starred in, from 1985 to 1989. You can use the keywords as provided in the given options and select t